**<center><font size = "5">Create custom NER model<center>**
***


In [ ]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.7/785.7 kB 10.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.3 MB/s eta 0:00:00a 0:00:01
  

In [ ]:
!python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.8 MB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from __future__ import unicode_literals, print_function

import pandas as pd
import os, time

from pathlib import Path
from tqdm import tqdm 
from spacy.training import Example
from spacy import displacy
import spacy
import base64

### NER pipeline

In [ ]:
nlpSpacy = spacy.load("en_core_web_lg")
 
print(nlpSpacy.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
file = open('sampletext.txt')
text = file.read()
file.close()
doc = nlpSpacy(text)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
for entity in doc.ents:
  print(f"Name of the entity {entity.text} type is {entity.label_}")

Name of the entity Electric type is ORG
Name of the entity Tesla type is ORG
Name of the entity Super Bowl type is EVENT
Name of the entity Sunday type is DATE
Name of the entity Autopilot type is ORG
Name of the entity Tesla type is ORG
Name of the entity Model S type is PRODUCT
Name of the entity Tesla type is ORG
Name of the entity Palo Alto type is GPE
Name of the entity California type is GPE
Name of the entity October 14, 2015 type is DATE
Name of the entity Tesla type is ORG
Name of the entity DOJ type is ORG
Name of the entity Autopilot type is ORG
Name of the entity Washington type is GPE
Name of the entity DC type is GPE
Name of the entity Austin type is GPE
Name of the entity Tallahassee type is GPE
Name of the entity Albany type is GPE
Name of the entity Atlanta type is GPE
Name of the entity Sacramento type is GPE
Name of the entity Tesla type is ORG
Name of the entity multimillion dollar type is MONEY
Name of the entity The Dawn Project type is ORG
Name of the entity Dan 

## Annotate the text for NER 

Navigate to 

https://tecoholic.github.io/ner-annotator/

and make all the custom NER and annotate the entities on the text one by one

## Make a Spacy custom NER MODEL

In [ ]:
import json
 
with open('annotations.json', 'r') as f:
    data = json.load(f)
    
print(data['annotations'][0])

['Electric carmaker Tesla will face a hit on Super Bowl Sunday, when an ad will play showing the alleged dangers of its Full Self-Driving technology.', {'entities': [[0, 17, 'SUPPLIER'], [18, 23, 'ORG'], [43, 53, 'EVENT'], [54, 60, 'DAY'], [123, 135, 'FEATURE']]}]


In [ ]:
data['classes']

['ORG',
 'PRODUCT',
 'MONEY',
 'TECHNOLOGY',
 'EVENT',
 'DAY',
 'LOCATION',
 'DATE',
 'DEPARTMENT',
 'FEATURE',
 'PERSON',
 'ROLE',
 'OBJECT',
 'PROJECT',
 'NUMBER',
 'CUSTOMERS',
 'BLOGS',
 'SUPPLIER']

## Create empty model

In [ ]:
training_data = data.copy()
training_data['classes'] = data['classes']
training_data['annotations'] = []
for text, annotation in data['annotations']:
    try:
        if text!="" and len(annotation['entities']) > 0:
            temp_dict = {}
            temp_dict['text'] = text
            temp_dict['entities'] = []
            for ent in annotation['entities']:
                start = ent[0]
                end = ent[1]
                label = ent[2].upper()
                temp_dict['entities'].append((start, end, label))
        training_data['annotations'].append(temp_dict)
    except:
        pass
print(training_data)

{'classes': ['ORG', 'PRODUCT', 'MONEY', 'TECHNOLOGY', 'EVENT', 'DAY', 'LOCATION', 'DATE', 'DEPARTMENT', 'FEATURE', 'PERSON', 'ROLE', 'OBJECT', 'PROJECT', 'NUMBER', 'CUSTOMERS', 'BLOGS', 'SUPPLIER'], 'annotations': [{'text': 'Electric carmaker Tesla will face a hit on Super Bowl Sunday, when an ad will play showing the alleged dangers of its Full Self-Driving technology.', 'entities': [(0, 17, 'SUPPLIER'), (18, 23, 'ORG'), (43, 53, 'EVENT'), (54, 60, 'DAY'), (123, 135, 'FEATURE')]}, {'text': 'The Autopilot features demonstrated in a Tesla Model S during a Tesla event in Palo Alto, California October 14, 2015. ', 'entities': [(4, 13, 'FEATURE'), (41, 46, 'ORG'), (47, 54, 'PRODUCT'), (64, 69, 'ORG'), (79, 88, 'LOCATION'), (90, 100, 'LOCATION'), (101, 117, 'DATE')]}, {'text': "Tesla confirms DOJ has requested documents on Autopilot, 'Full Self-Driving'", 'entities': [(0, 5, 'ORG'), (15, 18, 'DEPARTMENT'), (46, 55, 'FEATURE'), (57, 75, 'FEATURE')]}, {'text': 'The commercial, which will be a

### Configuration variables

In [ ]:
modelSpacy = None
n_iter=100

### Load the model

In [ ]:
if modelSpacy is not None:
    nlp = spacy.load(modelSpacy)  
    print("Loaded model '%s'" % modelSpacy)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


### Set up pipeline

In [ ]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

### Train the NER model

In [ ]:
#getting all the entities
for annotations in training_data["annotations"]:
    for ent in annotations['entities']:
        ner.add_label(ent[2])
        
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        losses = {}
        for text, annotations in tqdm(data["annotations"]):
            try:
                if text!="":
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    #Update the model
                    nlp.update(
                        [example],  
                        drop=0.5,  
                        sgd=optimizer,
                        losses=losses)
            except:
                pass
        print(losses)


 80%|████████  | 20/25 [00:00<00:00, 19.23it/s]/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "On December 31, Musk replied to a tweet by @WholeM..." with entities "[[3, 14, 'DATE'], [16, 20, 'PERSON'], [44, 57, 'BL...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
100%|██████████| 25/25 [00:01<00:00, 19.85it/s]


{'ner': 748.4617953865836}


100%|██████████| 25/25 [00:01<00:00, 17.51it/s]


{'ner': 295.5354982531711}


100%|██████████| 25/25 [00:01<00:00, 19.32it/s]


...

{'ner': 59.07130119329136}


100%|██████████| 25/25 [00:00<00:00, 26.05it/s]


{'ner': 49.040806157288465}


100%|██████████| 25/25 [00:00<00:00, 26.20it/s]

{'ner': 53.742256340147144}


### Test the trained model

In [ ]:
for annotations in training_data["annotations"]:
    doc = nlp(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Electric carmaker', 'SUPPLIER'), ('Tesla', 'ORG'), ('Super Bowl', 'EVENT'), ('Sunday', 'DAY'), ('Self-Driving', 'FEATURE')]
Tokens [('Electric', 'SUPPLIER', 3), ('carmaker', 'SUPPLIER', 1), ('Tesla', 'ORG', 3), ('will', '', 2), ('face', '', 2), ('a', '', 2), ('hit', '', 2), ('on', '', 2), ('Super', 'EVENT', 3), ('Bowl', 'EVENT', 1), ('Sunday', 'DAY', 3), (',', '', 2), ('when', '', 2), ('an', '', 2), ('ad', '', 2), ('will', '', 2), ('play', '', 2), ('showing', '', 2), ('the', '', 2), ('alleged', '', 2), ('dangers', '', 2), ('of', '', 2), ('its', '', 2), ('Full', '', 2), ('Self', 'FEATURE', 3), ('-', 'FEATURE', 1), ('Driving', 'FEATURE', 1), ('technology', '', 2), ('.', '', 2)]
Entities [('Autopilot', 'FEATURE'), ('Tesla', 'ORG'), ('Model S', 'PRODUCT'), ('Tesla', 'ORG'), ('Palo Alto', 'LOCATION'), ('California', 'LOCATION'), ('October 14, 2015', 'DATE')]
Tokens [('The', '', 2), ('Autopilot', 'FEATURE', 3), ('features', '', 2), ('demonstrated', '', 2), ('in', '', 2), ('a', ''

### Save the cusom NER model

In [ ]:
output_dir= os.path.abspath(os.getcwd())
nlp.to_disk(output_dir)
print("Saved model to", output_dir) 

Saved model to /content


### Test the saved custom model

In [ ]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for annotations in training_data["annotations"][3:5]:
    doc = nlp2(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /content
Entities [('Washington', 'LOCATION'), ('DC', 'LOCATION'), ('Austin', 'LOCATION'), ('Tallahassee', 'LOCATION'), ('Albany', 'LOCATION'), ('Atlanta', 'LOCATION'), ('Sacramento', 'LOCATION'), ('Tesla', 'ORG'), ('best light.', 'FEATURE'), ('multimillion dollar', 'MONEY'), ('advertising', 'DEPARTMENT'), ('Dawn Project.', 'DEPARTMENT'), ('Dan O’Dowd', 'PERSON'), ('California', 'LOCATION'), ('CEO', 'ROLE'), ('US Senate', 'DEPARTMENT')]
Tokens [('The', '', 2), ('commercial', '', 2), (',', '', 2), ('which', '', 2), ('will', '', 2), ('be', '', 2), ('aired', '', 2), ('in', '', 2), ('Washington', 'LOCATION', 3), (',', '', 2), ('DC', 'LOCATION', 3), (',', '', 2), ('Austin', 'LOCATION', 3), (',', '', 2), ('Tallahassee', 'LOCATION', 3), (',', '', 2), ('Albany', 'LOCATION', 3), (',', '', 2), ('Atlanta', 'LOCATION', 3), ('and', '', 2), ('Sacramento', 'LOCATION', 3), ('does', '', 2), ('not', '', 2), ('paint', '', 2), ('Tesla', 'ORG', 3), ('in', '', 2), ('the', '', 2), ('best', 'FEAT

In [ ]:
file = open('sampletext.txt')
text = file.read()
file.close()
nlp2 = spacy.load(output_dir)
doc = nlp2(text)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
with open('Restaurant_Reviews.tsv', 'r') as file:
    lines = file.readlines()

lines.pop(0)
data = []
for line in lines:
    text, label = line.strip().split('\t')
    data.append((text, int(label)))

In [ ]:
df = pd.DataFrame(data, columns=['text', 'label'])

# Check the first few rows of the DataFrame
print(df.head())

                                                text  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


In [ ]:
train_x = df.text

In [ ]:
train_x.to_csv('output.txt', sep='\t', index=False, header=False)

In [ ]:
file = open('output.txt')
text = file.read()
file.close()
doc = nlpSpacy(text)
displacy.render(doc, style="ent", jupyter=True)